# Importing Data

### Importing Output

In [1]:
import numpy as np
import pandas as pd

# import txt as a dataframe

df = pd.read_csv('output.txt', sep=',',header=0)
df.drop(['state_id'], axis=1, inplace=True)

#divide the dataframe in a list of dataframes, one for each time step

df_list = [df[df['time'] == i] for i in df['time'].unique()]

# drop the time column

for i in range(len(df_list)):
    df_list[i].drop(['time'], axis=1, inplace=True)

# for every time step, divide the dataframe in a list of dataframes, one for each production step

for i in range(len(df_list)):
    df_list[i] = [df_list[i][df_list[i]['production_step'] == j] for j in df_list[i]['production_step'].unique()]

# drop the production_step column

for i in range(len(df_list)):
    for j in range(len(df_list[i])):
        df_list[i][j].drop(['production_step'], axis=1, inplace=True)

# make the vertex column the index

for i in range(len(df_list)):
    for j in range(len(df_list[i])):
        df_list[i][j].set_index('vertex', inplace=True)

# transpose the dataframes

for i in range(len(df_list)):
    for j in range(len(df_list[i])):
        df_list[i][j] = df_list[i][j].T

c:\Users\alexa\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Importing Event_Log

In [2]:
import copy

# import txt as a dataframe

df = pd.read_csv('log.txt', sep=',',header=0)

#get the last time

last_time = df['time'].max()

#get the last node

last_node = 0
for i in range(len(df_list[0])):
    for j in range(len(df_list[0][i].columns)):
        if df_list[0][i].columns[j] > last_node:
            last_node = df_list[0][i].columns[j]
            
# make a list of dictionary with all the nodes for the product ids

tokens_list = []
tokens = {i:[] for i in range(last_node+1)}

for i in range(last_time):
    ndf = df[df['time'] == i+1]
    for j in range(len(ndf)):
        if ndf.iloc[j]['actual_node'] != -1:
            tokens[ndf.iloc[j]['actual_node']].append(ndf.iloc[j]['product_id'])
            if ndf.iloc[j]['previous_node'] != -1:
                tokens[ndf.iloc[j]['previous_node']].remove(ndf.iloc[j]['product_id'])
        else:
            tokens[ndf.iloc[j]['previous_node']].remove(ndf.iloc[j]['product_id'])
    tokens_list.append(copy.deepcopy(tokens))

# Visual Interface

In [3]:
# make a visual interface to show the dataframes using pygame

import pygame

pygame.init()

class Vertex:
    def __init__(self, x, y, r, color, id, percentage, products):
        self.x = x
        self.y = y
        self.r = r
        self.color = color
        self.id = id
        self.percentage = percentage
        self.products = products

    def draw(self, win):
        pygame.draw.circle(win, self.color, (self.x, self.y), self.r)
        font = pygame.font.SysFont('Arial', 20)
        if self.id < 10:
            text = font.render("0"+str(self.id), 1, (0,0,0))
        else:
            text = font.render(str(self.id), 1, (0,0,0))
        win.blit(text, (self.x - self.r/2+1, self.y-self.r/2-3))
        #draw a percentage bar
        pygame.draw.rect(win, (0,0,0), (self.x - self.r, self.y + self.r+5, 2*self.r, 10))
        pygame.draw.rect(win, (0,255,0), (self.x - self.r, self.y + self.r+5, 2*self.r*self.percentage, 10))

        #write the products below the vertex

        font = pygame.font.SysFont('Arial', 15)
        for i in range(len(self.products)):
            if self.products[i] < 10:
                text = font.render("0"+str(self.products[i]), 1, (0,0,0))
            else:
                text = font.render(str(self.products[i]), 1, (0,0,0))
            win.blit(text, (self.x - self.r - 10 + i*22, self.y + self.r + 15))

# set the screen size
screen = pygame.display.set_mode((1000, 800))

# set the screen title
pygame.display.set_caption("Factory Model")

# set the screen background
screen.fill((255, 255, 255))

timeStamp = 0
maxTS = len(df_list) - 1
numSteps = len(df_list[0])
tick = 0
autoRun = False

steps = len(df_list[0])

clock = pygame.time.Clock()
run = True
while run:
    screen.fill((255, 255, 255))
    clock.tick(60)
    tick += 1

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False 

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RIGHT:
                timeStamp += 1
                if timeStamp > maxTS:
                    timeStamp = maxTS
            if event.key == pygame.K_LEFT:
                timeStamp -= 1
                if timeStamp < 0:
                    timeStamp = 0
            if event.key == pygame.K_SPACE:
                autoRun = not autoRun
            if event.key == pygame.K_BACKSPACE:
                timeStamp = 0
                autoRun = False
            if event.key == pygame.K_ESCAPE:
                run = False
        
    if autoRun:
        if tick % 1 == 0:
            timeStamp += 1
        if timeStamp > maxTS:
            timeStamp = 0
    
    #display the time step
    font = pygame.font.SysFont('Arial', 30)
    text = font.render("Time Step: " + str(timeStamp), 1, (0,0,0))
    screen.blit(text, (10,10))

    # generate the vertices

    vertices = []
    numVertices = len(df_list[timeStamp])-1
    for i in range(len(df_list[timeStamp])):
        numVerticesLevel = len(df_list[timeStamp][i].columns)-1
        y = 100 + int(600/numVertices * i)
        #draw the text of the step
        font = pygame.font.SysFont('Arial', 25)
        text = font.render("Step: "+str(numSteps-i), 1, (0,0,0))
        screen.blit(text, (50, y))
        for j in range(len(df_list[timeStamp][i].columns)):
            idi = df_list[timeStamp][i].columns[j]
            state = df_list[timeStamp][i].iloc[0,j]
            porcentage = df_list[timeStamp][i].iloc[1,j]
            if state == "working":
                color = (0,255,0)
            elif state == "starved":
                color = (100,100,100)
            elif state == "blocked":
                color = (255,0,0)
            else:
                color = (255,255,0)

            x = 200 + int(700/numVerticesLevel * j)

            vertices.append(Vertex(x, y, 20, color, idi, porcentage, tokens_list[timeStamp][idi])) 
    
    for vertex in vertices:
        vertex.draw(screen)
    
    #release vertices memory
    vertices = []

    pygame.display.update()        

pygame.quit()

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
